In [1]:
import gradio as gr
import re
import requests
import json
import base64
import os
from openai import OpenAI

In [2]:
client = OpenAI(
base_url = "https://openrouter.ai/api/v1",
api_key = "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"
)

In [3]:
def encode_pdf_to_base64(pdf_path):
    with open(pdf_path, "rb") as pdf_file:
        return base64.b64encode(pdf_file.read()).decode('utf-8')
url = "https://openrouter.ai/api/v1/chat/completions"
headers = {
    "Authorization": f"Bearer {"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"}",
    "Content-Type": "application/json"
}

In [4]:
messages = []

In [5]:
def chatbot_stream(user_input, pdf_file=None):
    global messages
    content = [{"type": "text", "text": user_input}]
    messages.append({"role": "user", "content": content})
    # API Call
    response = client.chat.completions.create(
        model="meta-llama/llama-4-maverick:free",
        messages=messages,
        stream=True,
        extra_body={
            "plugins": [
                {"id": "web"},
                {"id": "file-parser", "pdf": {"engine": "pdf-text"}}
            ]
        }
    )

    # Collect streamed response
    bot_reply = ""
    for chunk in response:
        if hasattr(chunk.choices[0].delta, 'content'):
            bot_reply += chunk.choices[0].delta.content

    messages.append({"role": "assistant", "content": bot_reply})
    return bot_reply

In [ ]:
demo = gr.Interface(
    fn=chatbot_stream,
    inputs=[
        gr.Textbox(label="User Input"),  # User's text input
        gr.File(label="Upload PDF (optional)")  # Optional PDF file input
    ],
    outputs=gr.Textbox(label="Bot Reply"),  # Bot's response
    title="Chatbot with PDF Upload",
    theme="default",
    flagging_mode="never",
)

In [ ]:
demo.launch(share=True, server_name="Test_nang_luc")